In [1]:
# force install packages 
%pip install pandas
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import pandas as pd
from io import BytesIO

# Set display options to show all columns
pd.set_option('display.max_columns', None)

In [3]:
# Year 2022
# URL of the Excel file
url = 'https://www.acquisition.gov/sites/default/files/page_file_uploads/FY22_GOVERNMENT_INVENTORY.xlsx'

# Send a GET request to the URL
response = requests.get(url) # change to internet url after testing ingest
response.raise_for_status()  # Ensure download succeeded

# Read the downloaded Excel file content into a pandas DataFrame
# BytesIO allows the response content to be read as if it were a file
df22_raw = pd.read_excel(BytesIO(response.content))

In [4]:
# Display the shape of the dataframe
df22_raw.shape

(80560, 422)

In [5]:
## Data Wrangling ##
# Duplicate the data for wrangling
df22 = df22_raw.copy()

In [6]:
%whos

Variable   Type         Data/Info
---------------------------------
BytesIO    type         <class '_io.BytesIO'>
df22       DataFrame           Unnamed: 0        <...>80560 rows x 422 columns]
df22_raw   DataFrame           Unnamed: 0        <...>80560 rows x 422 columns]
pd         module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests   module       <module 'requests' from '<...>\\requests\\__init__.py'>
response   Response     <Response [200]>
url        str          https://www.acquisition.g<...>GOVERNMENT_INVENTORY.xlsx


In [7]:
# Delete the first 3 rows as they are not data
df22 = df22.iloc[2:]
%whos

Variable   Type         Data/Info
---------------------------------
BytesIO    type         <class '_io.BytesIO'>
df22       DataFrame           Unnamed: 0 Unnamed<...>80558 rows x 422 columns]
df22_raw   DataFrame           Unnamed: 0        <...>80560 rows x 422 columns]
pd         module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests   module       <module 'requests' from '<...>\\requests\\__init__.py'>
response   Response     <Response [200]>
url        str          https://www.acquisition.g<...>GOVERNMENT_INVENTORY.xlsx


In [8]:
#df22

In [9]:
# Reset the index
df22.reset_index(drop=True, inplace=True)
#df22

In [10]:
# Set the new 1st row as the column names
df22.columns = df22.iloc[0]
#df22

In [11]:
# Drop the first row with index 0
df22.drop(0, inplace=True)
#df22

In [12]:
# Reset the index again
df22.reset_index(drop=True, inplace=True)

# Drop the first column
df22.drop(df22.columns[0], axis=1, inplace=True)
#df22

In [13]:
# Reset the index again
df22.reset_index(drop=True, inplace=True)
#df22

In [14]:
#Save the list of column names to a list
column_names_list = df22.columns.tolist()

In [15]:
# print(column_names_list)

In [16]:
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=421
df22                DataFrame    0     PSC Code           <...>80557 rows x 421 columns]
df22_raw            DataFrame           Unnamed: 0        <...>80560 rows x 422 columns]
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module 'requests' from '<...>\\requests\\__init__.py'>
response            Response     <Response [200]>
url                 str          https://www.acquisition.g<...>GOVERNMENT_INVENTORY.xlsx


In [17]:
# Trim leading and trailing spaces from column names
# This was a problem with some of the datasets, the IGF feature had trailing spaces and would not load
df22.columns = df22.columns.str.strip()

In [18]:
# Now drop all columns except those that will be used in the regression analysis
# List of columns to keep
columns_to_keep = ['PSC Code', 'PSC Description', 'Funding Agency Name', 'Place of Performance Country', 'Date Signed', 'Extent Competed', 'Type of Contract', 
                   'Description of Requirement', 'Inherently Governmental Functions', 'Vendor Name', 'Total Base and All Options Value']

# Drop all columns except those in columns_to_keep
columns_to_drop = [col for col in df22.columns if col not in columns_to_keep]
df22.drop(columns_to_drop, axis=1, inplace=True)
df22

,PSC Code,PSC Description,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,D317,IT AND TELECOM- WEB-BASED SUBSCRIPTION,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2022-05-27,COMPETED UNDER SAP,FIRM FIXED PRICE,DATABASE,OTHER FUNCTIONS,RELX INC.,6911482.07
1,B506,SPECIAL STUDIES/ANALYSIS- DATA (OTHER THAN SCI...,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2022-08-04,NOT AVAILABLE FOR COMPETITION,FIXED PRICE LEVEL OF EFFORT,BUSINESS SUPPORT SERVICES,CLOSELY ASSOCIATED,"NORTH WIND INFRASTRUCTURE AND TECHNOLOGY, LLC",10864820.26
2,AN11,HEALTH R&D SERVICES; HEALTH CARE SERVICES; BAS...,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2021-12-20,NOT COMPETED UNDER SAP,COST NO FEE,WORKSHOP ON METHADONE TREATMENT FOR OPIOID USE...,OTHER FUNCTIONS,NATIONAL ACADEMY OF SCIENCES,299163
3,B539,SPECIAL STUDIES/ANALYSIS- AERONAUTICAL/SPACE,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2022-06-15,NOT COMPETED UNDER SAP,LABOR HOURS,TECHNICAL STUDIES AND ANALYTICS SUPPORT SERVICES,OTHER FUNCTIONS,"BRYCE SPACE AND TECHNOLOGY, LLC",726371.02
4,B544,SPECIAL STUDIES/ANALYSIS- TECHNOLOGY,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2022-12-28,NOT COMPETED,COST PLUS FIXED FEE,BUDGET REVIEW,OTHER FUNCTIONS,JOHNS HOPKINS UNIVERSITY APPLIED PHYSICS LABOR...,1200000
...,...,...,...,...,...,...,...,...,...,...,...
80552,DA10,IT AND TELECOM - BUSINESS APPLICATION/APPLICAT...,DEFENSE INFORMATION SYSTEMS AGENCY (DISA),USA,2022-09-22,COMPETED,NaN,OFFICE OF OPEN LEARNING,NaN,"APPLIED RESEARCH INSTITUTE, INC.",8147500
80553,DA10,IT AND TELECOM - BUSINESS APPLICATION/APPLICAT...,DEFENSE INFORMATION SYSTEMS AGENCY (DISA),USA,2022-09-16,COMPETED,NaN,AI LITERACY - IN3,NaN,"APPLIED RESEARCH INSTITUTE, INC.",3783400
80554,DA10,IT AND TELECOM - BUSINESS APPLICATION/APPLICAT...,DEFENSE INFORMATION SYSTEMS AGENCY (DISA),USA,2022-09-16,COMPETED,NaN,AI AUXILIARY PILOT,NaN,CYBER BYTES FOUNDATION,3272258
80555,DA10,IT AND TELECOM - BUSINESS APPLICATION/APPLICAT...,DEFENSE INFORMATION SYSTEMS AGENCY (DISA),USA,2022-09-27,COMPETED,NaN,DIGITAL PROVING GROUND,NaN,"ADVANCED TECHNOLOGY ACADEMIC RESEARCH CENTER, ...",4872000


In [19]:
# Add a new first column with value '2022'
df22.insert(0, 'Year_Reported', '2022')
#df22

In [20]:
# Reset the indexes
df22.reset_index(drop=True, inplace=True)
#df22

In [21]:
# clean up memory
del df22_raw
del url
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=421
columns_to_drop     list         n=410
columns_to_keep     list         n=11
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module 'requests' from '<...>\\requests\\__init__.py'>
response            Response     <Response [200]>


## Ingest Year 2021

In [22]:
# URL of the Excel file
url = 'https://www.acquisition.gov/sites/default/files/page_file_uploads/Final%20Report-GOVERNMENT%20INVENTORY-Civilian-FY2021_20220215.xlsx'

# Send a GET request to the URL
response = requests.get(url) # change to internet url after testing ingest
response.raise_for_status()  # Ensure download succeeded

# Read the downloaded Excel file content into a pandas DataFrame
# BytesIO allows the response content to be read as if it were a file
df21_raw = pd.read_excel(BytesIO(response.content))
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=421
columns_to_drop     list         n=410
columns_to_keep     list         n=11
df21_raw            DataFrame           Unnamed: 0        <...>74589 rows x 436 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module 'requests' from '<...>\\requests\\__init__.py'>
response            Response     <Response [200]>
url                 str          https://www.acquisition.g<...>lian-FY2021_20220215.xlsx


In [23]:
# Year 2021

## Data Wrangling ##
# Duplicate the data for wrangling
df21 = df21_raw.copy()
# Delete the first 3 rows as they are not data
df21 = df21.iloc[2:]
# Reset the index
df21.reset_index(drop=True, inplace=True)
# Set the new 1st row as the column names
df21.columns = df21.iloc[0]
# Drop the first row with index 0
df21.drop(0, inplace=True)
# Reset the index again
df21.reset_index(drop=True, inplace=True)
# Drop the first column
df21.drop(df21.columns[0], axis=1, inplace=True)
# Reset the index again
df21.reset_index(drop=True, inplace=True)
#Save the list of column names to a list
column_names_list = df21.columns.tolist()
# Now drop all columns except those that will be used in the regression analysis
# Trim leading and trailing spaces from column names
# This was a problem with some of the datasets, the IGF feature had trailing spaces and would not load
df21.columns = df21.columns.str.strip()
# Drop all columns except those in columns_to_keep
columns_to_drop = [col for col in df21.columns if col not in columns_to_keep]
df21.drop(columns_to_drop, axis=1, inplace=True)
# Add a new first column with value '2021'
df21.insert(0, 'Year_Reported', '2021')
# Reset the indexes
df21.reset_index(drop=True, inplace=True)
df21

,Year_Reported,PSC Code,PSC Description,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,2021,D399,IT AND TELECOM- OTHER IT AND TELECOMMUNICATIONS,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2020-09-10,NOT COMPETED UNDER SAP,FIRM FIXED PRICE,ADMIN MOD - COR CHANGE,OTHER FUNCTIONS,"CHENEGA IT ENTERPRISE SERVICES, LLC",5066977.19
1,2021,D317,IT AND TELECOM- WEB-BASED SUBSCRIPTION,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2021-04-07,COMPETED UNDER SAP,FIRM FIXED PRICE,DATABASE - FUNDING FOR OPTION YEAR THREE,OTHER FUNCTIONS,RELX INC.,5385173.07
2,2021,D318,IT AND TELECOM- INTEGRATED HARDWARE/SOFTWARE/S...,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2021-05-18,NOT AVAILABLE FOR COMPETITION,FIRM FIXED PRICE,OTHER ADMINISTRATIVE ACTION,OTHER FUNCTIONS,"FEARLESS SOLUTIONS, LLC",1687203.12
3,2021,B506,SPECIAL STUDIES/ANALYSIS- DATA (OTHER THAN SCI...,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2021-08-19,NOT AVAILABLE FOR COMPETITION,FIXED PRICE LEVEL OF EFFORT,ADMINISTRATIVE ACTION TO UPDATE COR AND ACOR,CLOSELY ASSOCIATED,"NORTH WIND INFRASTRUCTURE AND TECHNOLOGY, LLC",10864820.26
4,2021,AN11,HEALTH R&D SERVICES; HEALTH CARE SERVICES; BAS...,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2021-09-13,NOT COMPETED UNDER SAP,COST NO FEE,WORKSHOP ON METHADONE TREATMENT FOR OPIOID USE...,OTHER FUNCTIONS,NATIONAL ACADEMY OF SCIENCES,299163
...,...,...,...,...,...,...,...,...,...,...,...,...
74581,2021,F999,OTHER ENVIRONMENTAL SERVICES,ENVIRONMENTAL PROTECTION AGENCY,USA,2021-05-28,NOT COMPETED,FIRM FIXED PRICE,ENVIRONMENTAL SERVICES,OTHER FUNCTIONS,"ARDL, INC.",963908.97
74582,2021,F999,OTHER ENVIRONMENTAL SERVICES,ENVIRONMENTAL PROTECTION AGENCY,USA,2021-06-03,NOT COMPETED,FIRM FIXED PRICE,ENVIRONMENTAL SERVICES,OTHER FUNCTIONS,"ARDL, INC.",913419.12
74583,2021,F999,OTHER ENVIRONMENTAL SERVICES,ENVIRONMENTAL PROTECTION AGENCY,USA,2021-06-03,NOT COMPETED,FIRM FIXED PRICE,ENVIRONMENTAL SERVICES,OTHER FUNCTIONS,"ARDL, INC.",1746713.94
74584,2021,C219,ARCHITECT AND ENGINEERING- GENERAL: OTHER,ENVIRONMENTAL PROTECTION AGENCY,USA,2020-10-15,FULL AND OPEN COMPETITION AFTER EXCLUSION OF S...,FIRM FIXED PRICE,"INFRASTRUCTURE DESIGN RAYMARK SUPERFUND SITE, ...",OTHER FUNCTIONS,MINUTEMAN DESIGN VENTURES LLC,572114.52


In [24]:
# clean up memory
del df21_raw
del url
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=435
columns_to_drop     list         n=424
columns_to_keep     list         n=11
df21                DataFrame    0     Year_Reported PSC C<...>[74586 rows x 12 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module 'requests' from '<...>\\requests\\__init__.py'>
response            Response     <Response [200]>


## Ingest Year 2020

In [25]:
# URL of the Excel file
url = 'https://www.acquisition.gov/sites/default/files/page_file_uploads/FY2020_Service_Contract_Inventory-Civilian-2-17-21.xlsx'
# Send a GET request to the URL
response = requests.get(url) # change to internet url after testing ingest
response.raise_for_status()  # Ensure download succeeded

# Read the downloaded Excel file content into a pandas DataFrame
# BytesIO allows the response content to be read as if it were a file
df20_raw = pd.read_excel(BytesIO(response.content))
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=435
columns_to_drop     list         n=424
columns_to_keep     list         n=11
df20_raw            DataFrame            Unnamed: 0       <...>14346 rows x 476 columns]
df21                DataFrame    0     Year_Reported PSC C<...>[74586 rows x 12 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module 'requests' from '<...>\\requests\\__init__.py'>
response            Response     <Response [200]>
url                 str          https://www.acquisition.g<...>ory-Civilian-2-17-21.xlsx


In [26]:
#df20_raw

In [27]:
# Year 2020

## Data Wrangling ##
# Duplicate the data for wrangling
df20 = df20_raw.copy()
# Delete the first 4 rows as they are not data
df20 = df20.iloc[4:]
df20
# Reset the index
df20.reset_index(drop=True, inplace=True)
# Set the new 1st row as the column names
df20.columns = df20.iloc[0]
# Drop the first row with index 0
df20.drop(0, inplace=True)
# Reset the index again
df20.reset_index(drop=True, inplace=True)
# Drop the first column
df20.drop(df20.columns[0], axis=1, inplace=True)
# Reset the index again
df20.reset_index(drop=True, inplace=True)
#Save the list of column names to a list
column_names_list = df20.columns.tolist()
# Now drop all columns except those that will be used in the regression analysis
# Trim leading and trailing spaces from column names
# This was a problem with some of the datasets, the IGF feature had trailing spaces and would not load
df20.columns = df20.columns.str.strip()
# Drop all columns except those in columns_to_keep
columns_to_drop = [col for col in df20.columns if col not in columns_to_keep]
df20.drop(columns_to_drop, axis=1, inplace=True)
# Add a new first column with value '2020'
df20.insert(0, 'Year_Reported', '2020')
# Reset the indexes
df20.reset_index(drop=True, inplace=True)
df20

,Year_Reported,PSC Code,PSC Description,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,2020,D399,IT AND TELECOM- OTHER IT AND TELECOMMUNICATIONS,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2020-09-10 00:00:00,NOT COMPETED UNDER SAP,FIRM FIXED PRICE,ADMIN MOD - COR CHANGE,OTHER FUNCTIONS,"CHENEGA IT ENTERPRISE SERVICES, LLC",5066977.19
1,2020,D317,IT AND TELECOM- WEB-BASED SUBSCRIPTION,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2020-08-26 00:00:00,COMPETED UNDER SAP,FIRM FIXED PRICE,DATABASE,OTHER FUNCTIONS,RELX INC.,3903316.07
2,2020,D318,IT AND TELECOM- INTEGRATED HARDWARE/SOFTWARE/S...,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2020-03-25 00:00:00,NOT AVAILABLE FOR COMPETITION,FIRM FIXED PRICE,PRODUCT OWNER TRAINING PILOT FOR USDS,OTHER FUNCTIONS,"FEARLESS SOLUTIONS, LLC",1538009
3,2020,B506,SPECIAL STUDIES/ANALYSIS- DATA (OTHER THAN SCI...,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2020-08-26 00:00:00,NOT AVAILABLE FOR COMPETITION,FIXED PRICE LEVEL OF EFFORT,MODIFICATION TO REQUISITION FOR NORTH WIND INF...,CLOSELY ASSOCIATED,"NORTH WIND INFRASTRUCTURE AND TECHNOLOGY, LLC",10864820.26
4,2020,D399,IT AND TELECOM- OTHER IT AND TELECOMMUNICATIONS,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2020-01-23 00:00:00,NOT COMPETED,FIRM FIXED PRICE,"CACI BRIDGE FOR OA, NSC AND WHCA",OTHER FUNCTIONS,"CACI, INC. - FEDERAL",8317503.2
...,...,...,...,...,...,...,...,...,...,...,...,...
114336,2020,Y1KZ,CONSTRUCTION OF OTHER CONSERVATION AND DEVELOP...,ENVIRONMENTAL PROTECTION AGENCY,USA,2020-09-29 00:00:00,FULL AND OPEN COMPETITION AFTER EXCLUSION OF S...,FIRM FIXED PRICE,MODIFICATION TO CUT ADDITIONAL TREES AND EXTEN...,OTHER FUNCTIONS,"ANGLIN CIVIL, LLC",1252536.3
114337,2020,F108,ENVIRONMENTAL SYSTEMS PROTECTION- ENVIRONMENTA...,ENVIRONMENTAL PROTECTION AGENCY,USA,2020-09-29 00:00:00,NOT AVAILABLE FOR COMPETITION,COST PLUS FIXED FEE,HUDSON RIVER AWS FUNDING MOD P0 0001,OTHER FUNCTIONS,ACT SERVICES LLC,902658.2
114338,2020,Y1PZ,CONSTRUCTION OF OTHER NON-BUILDING FACILITIES,FEDERAL EMERGENCY MANAGEMENT AGENCY,USA,2020-05-18 00:00:00,FULL AND OPEN COMPETITION,FIRM FIXED PRICE,COVID-19 ALTERNATE CARE FACILITY (ACF) HAGERST...,"CLOSELY ASSOCIATED,CRITICAL FUNCTIONS","CLARK CONSTRUCTION GROUP, LLC",2637590
114339,2020,C1DZ,ARCHITECT AND ENGINEERING- CONSTRUCTION: OTHER...,FEDERAL EMERGENCY MANAGEMENT AGENCY,USA,2020-05-11 00:00:00,NOT COMPETED,FIRM FIXED PRICE,RETROFIT INTO ALTERNATE CARE FACILITY - VAN CO...,OTHER FUNCTIONS,PARSONS CORPORATION,700000


In [28]:
# clean up memory
del df20_raw
del url
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=475
columns_to_drop     list         n=464
columns_to_keep     list         n=11
df20                DataFrame    0      Year_Reported PSC <...>114341 rows x 12 columns]
df21                DataFrame    0     Year_Reported PSC C<...>[74586 rows x 12 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module 'requests' from '<...>\\requests\\__init__.py'>
response            Response     <Response [200]>


## Ingest Year 2019

In [29]:
# URL of the Excel file
url = 'https://www.acquisition.gov/sites/default/files/page_file_uploads/FY19_Final-Report-GOVERNMENT_INVENTORY_20200214.xlsx'
# Send a GET request to the URL
response = requests.get(url) # change to internet url after testing ingest
response.raise_for_status()  # Ensure download succeeded

# Read the downloaded Excel file content into a pandas DataFrame
# BytesIO allows the response content to be read as if it were a file
df19_raw = pd.read_excel(BytesIO(response.content))
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=475
columns_to_drop     list         n=464
columns_to_keep     list         n=11
df19_raw            DataFrame           Unnamed: 0        <...>96173 rows x 476 columns]
df20                DataFrame    0      Year_Reported PSC <...>114341 rows x 12 columns]
df21                DataFrame    0     Year_Reported PSC C<...>[74586 rows x 12 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module 'requests' from '<...>\\requests\\__init__.py'>
response            Response     <Response [200]>
url                 str          https://www.acquisition.g<...>T_INVENTORY_20200214.xlsx


In [30]:
# Year 2019

## Data Wrangling ##
# Duplicate the data for wrangling
df19 = df19_raw.copy()
# Delete the first 4 rows as they are not data
df19 = df19.iloc[4:]
# Reset the index
df19.reset_index(drop=True, inplace=True)
# Set the new 1st row as the column names
df19.columns = df19.iloc[0]
# Drop the first row with index 0
df19.drop(0, inplace=True)
# Reset the index again
df19.reset_index(drop=True, inplace=True)
# Drop the first column
df19.drop(df19.columns[0], axis=1, inplace=True)
# Reset the index again
df19.reset_index(drop=True, inplace=True)
#Save the list of column names to a list
column_names_list = df19.columns.tolist()
# Now drop all columns except those that will be used in the regression analysis
# Trim leading and trailing spaces from column names
# This was a problem with some of the datasets, the IGF feature had trailing spaces and would not load
df19.columns = df19.columns.str.strip()
# Drop all columns except those in columns_to_keep
columns_to_drop = [col for col in df19.columns if col not in columns_to_keep]
df19.drop(columns_to_drop, axis=1, inplace=True)
# Add a new first column with value '2019'
df19.insert(0, 'Year_Reported', '2019')
# Reset the indexes
df19.reset_index(drop=True, inplace=True)
df19

,Year_Reported,PSC Code,PSC Description,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,2019,D399,IT AND TELECOM- OTHER IT AND TELECOMMUNICATIONS,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2019-08-02,NOT COMPETED UNDER SAP,FIRM FIXED PRICE,SUPPORT,OTHER FUNCTIONS,"CHENEGA IT ENTERPRISE SERVICES, LLC",5202744.6
1,2019,D317,IT AND TELECOM- WEB-BASED SUBSCRIPTION,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2018-12-21,COMPETED UNDER SAP,FIRM FIXED PRICE,DATABASEIGF::OT::IGF FOR OTHER FUNCTIONS,OTHER FUNCTIONS,RELX INC.,2465764.07
2,2019,Y1FZ,CONSTRUCTION OF OTHER RESIDENTIAL BUILDINGS,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2019-07-26,NOT COMPETED,FIRM FIXED PRICE,TO PROVIDE FUNDING FOR A CHANGE ORDER UNDER EX...,OTHER FUNCTIONS,"WHITING-TURNER CONTRACTING COMPANY, THE",1244392.49
3,2019,D318,IT AND TELECOM- INTEGRATED HARDWARE/SOFTWARE/S...,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2019-09-23,NOT AVAILABLE FOR COMPETITION,FIRM FIXED PRICE,PRODUCT OWNER TRAINING PILOT FOR USDS,OTHER FUNCTIONS,"FEARLESS SOLUTIONS, LLC",1531009
4,2019,B506,SPECIAL STUDIES/ANALYSIS- DATA (OTHER THAN SCI...,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2019-08-07,NOT AVAILABLE FOR COMPETITION,FIXED PRICE LEVEL OF EFFORT,301 INVESTIGATIONS EXCLUSION PROGRAM SUPPORT,CLOSELY ASSOCIATED,"NORTH WIND INFRASTRUCTURE AND TECHNOLOGY, LLC",13809344.71
...,...,...,...,...,...,...,...,...,...,...,...,...
96163,2019,Q503,MEDICAL- DENTISTRY,"VETERANS AFFAIRS, DEPARTMENT OF",USA,2019-11-15,NOT COMPETED,FIRM FIXED PRICE,DENTAL LAB SERVICES FOR VA SOUTHERN NEVADA HEA...,OTHER FUNCTIONS,ACRYLIC WORKS DENTAL LAB,460093.5
96164,2019,J044,"MAINT/REPAIR/REBUILD OF EQUIPMENT- FURNACE, ST...","VETERANS AFFAIRS, DEPARTMENT OF",USA,2019-10-09,FULL AND OPEN COMPETITION AFTER EXCLUSION OF S...,FIRM FIXED PRICE,"IMPROVE STEAM DISTRIBUTION SYSTEM, BLDG 256 VA...",OTHER FUNCTIONS,"J K TECHNICAL SERVICE, INC",1046312
96165,2019,C1AZ,ARCHITECT AND ENGINEERING- CONSTRUCTION: OTHER...,"VETERANS AFFAIRS, DEPARTMENT OF",USA,2018-04-24,NOT COMPETED,FIRM FIXED PRICE,IGF::OT::IGF-DESIGN BLDG 222 PARKING LOT&GROUNDS,OTHER FUNCTIONS,"ANDERSON ENGINEERING OF MINNESOTA, LLC",334502.5
96166,2019,S208,HOUSEKEEPING- LANDSCAPING/GROUNDSKEEPING,"VETERANS AFFAIRS, DEPARTMENT OF",USA,2019-10-24,FULL AND OPEN COMPETITION AFTER EXCLUSION OF S...,FIRM FIXED PRICE,TURF MAINTENANCE,OTHER FUNCTIONS,"ROBERT F. HYLAND & SONS, LLC",295751


In [31]:
# clean up memory
del df19_raw
del url
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=475
columns_to_drop     list         n=464
columns_to_keep     list         n=11
df19                DataFrame    0     Year_Reported PSC C<...>[96168 rows x 12 columns]
df20                DataFrame    0      Year_Reported PSC <...>114341 rows x 12 columns]
df21                DataFrame    0     Year_Reported PSC C<...>[74586 rows x 12 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module 'requests' from '<...>\\requests\\__init__.py'>
response            Response     <Response [200]>


## Ingest Year 2018

In [32]:
# URL of the Excel file
url = 'https://www.acquisition.gov/sites/default/files/page_file_uploads/FY18_GOVERNMENT_INVENTORY_REPORT.xlsx'
# Send a GET request to the URL
response = requests.get(url) # change to internet url after testing ingest
response.raise_for_status()  # Ensure download succeeded

# Read the downloaded Excel file content into a pandas DataFrame
# BytesIO allows the response content to be read as if it were a file
df18_raw = pd.read_excel(BytesIO(response.content))
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=475
columns_to_drop     list         n=464
columns_to_keep     list         n=11
df18_raw            DataFrame           Unnamed: 0        <...>78121 rows x 439 columns]
df19                DataFrame    0     Year_Reported PSC C<...>[96168 rows x 12 columns]
df20                DataFrame    0      Year_Reported PSC <...>114341 rows x 12 columns]
df21                DataFrame    0     Year_Reported PSC C<...>[74586 rows x 12 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module 'requests' from '<...>\\requests\\__init__.py'>
response            Response     <Response [200]>
url                 str          https://www.acquisition.g<...>ENT_INV

In [33]:
# Year 2018

## Data Wrangling ##
# Duplicate the data for wrangling
df18 = df18_raw.copy()
# Delete the first 4 rows as they are not data
df18 = df18.iloc[4:]
# Reset the index
df18.reset_index(drop=True, inplace=True)
# Set the new 1st row as the column names
df18.columns = df18.iloc[0]
# Drop the first row with index 0
df18.drop(0, inplace=True)
# Reset the index again
df18.reset_index(drop=True, inplace=True)
# Drop the first column
df18.drop(df18.columns[0], axis=1, inplace=True)
# Reset the index again
df18.reset_index(drop=True, inplace=True)
#Save the list of column names to a list
column_names_list = df18.columns.tolist()
# Now drop all columns except those that will be used in the regression analysis
# Trim leading and trailing spaces from column names
# This was a problem with some of the datasets, the IGF feature had trailing spaces and would not load
df18.columns = df18.columns.str.strip()
# Drop all columns except those in columns_to_keep
columns_to_drop = [col for col in df18.columns if col not in columns_to_keep]
df18.drop(columns_to_drop, axis=1, inplace=True)
# Add a new first column with value '2018'
df18.insert(0, 'Year_Reported', '2018')
# Reset the indexes
df18.reset_index(drop=True, inplace=True)
df18

,Year_Reported,PSC Code,PSC Description,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,2018,R425,SUPPORT- PROFESSIONAL: ENGINEERING/TECHNICAL,FEDERAL BUREAU OF INVESTIGATION,USA,09-28-2018,NOT COMPETED,FIRM FIXED PRICE,PROFESSIONAL TECHNICAL SUPPORT SERVICES,NONE,"SUVI GLOBAL SERVICES, LLC",12289962
1,2018,D310,IT AND TELECOM- CYBER SECURITY AND DATA BACKUP,FEDERAL BUREAU OF INVESTIGATION,USA,09-27-2018,NOT COMPETED,LABOR HOURS,IGF::OT::IGF CASTVIZ 3.0 SYSTEM,NONE,"CACI, INC. - FEDERAL",427544
2,2018,R424,SUPPORT- PROFESSIONAL: EXPERT WITNESS,"OFFICES, BOARDS AND DIVISIONS",USA,09-18-2018,NOT COMPETED UNDER SAP,TIME AND MATERIALS,MODIFICATION TO YIO JENRD1804334. ORIGINAL CO...,NONE,"INDUSTRIAL ECONOMICS, INCORPORATED",162946
3,2018,Z1AA,MAINTENANCE OF OFFICE BUILDINGS,"STATE, DEPARTMENT OF",USA,09-26-2018,NOT AVAILABLE FOR COMPETITION,TIME AND MATERIALS,FACILITIES MAINTENANCE SERVICES FOR THE FOREIG...,NONE,T&H SERVICES LLC,13341035
4,2018,R408,SUPPORT- PROFESSIONAL: PROGRAM MANAGEMENT/SUPPORT,"STATE, DEPARTMENT OF",USA,09-29-2018,NOT AVAILABLE FOR COMPETITION,TIME AND MATERIALS,THE MODIFICATION PROVIDES INCREMENTAL FUNDING....,S,"TUVA, LLC",19914437.87
...,...,...,...,...,...,...,...,...,...,...,...,...
78111,2018,B510,SPECIAL STUDIES/ANALYSIS- ENVIRONMENTAL ASSESS...,ENVIRONMENTAL PROTECTION AGENCY,USA,2018-05-01,NOT COMPETED,FIRM FIXED PRICE,SITE SURVEY WITH CORE SAMPLING FOR JUAN HORSE ...,NONE,"WESTON SOLUTIONS, INC.",225527.77
78112,2018,F999,OTHER ENVIRONMENTAL SERVICES,ENVIRONMENTAL PROTECTION AGENCY,USA,2018-01-17,NOT AVAILABLE FOR COMPETITION,FIRM FIXED PRICE,"IGF::OT::IGFCALLAHAN MINE O&M CONTRACT, MAINE.",NaN,"CREDERE ASSOCIATES, LLC",183407.06
78113,2018,C219,ARCHITECT AND ENGINEERING- GENERAL: OTHER,ENVIRONMENTAL PROTECTION AGENCY,USA,2018-10-04,FULL AND OPEN COMPETITION,FIRM FIXED PRICE,ARCHITECT ENGINEER PROJECT MANAGEMENT SERVICES...,NaN,"AECOM TECHNICAL SERVICES, INC.",2751982.33
78114,2018,R499,SUPPORT- PROFESSIONAL: OTHER,ENVIRONMENTAL PROTECTION AGENCY,USA,2018-09-14,FULL AND OPEN COMPETITION,FIRM FIXED PRICE,TECHNICAL SUPPORT FOR THE NEW BEDFORD HARBOR S...,NaN,"AECOM TECHNICAL SERVICES, INC.",11183960.43


In [34]:
# clean up memory
del df18_raw
del url
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=438
columns_to_drop     list         n=427
columns_to_keep     list         n=11
df18                DataFrame    0     Year_Reported PSC C<...>[78116 rows x 12 columns]
df19                DataFrame    0     Year_Reported PSC C<...>[96168 rows x 12 columns]
df20                DataFrame    0      Year_Reported PSC <...>114341 rows x 12 columns]
df21                DataFrame    0     Year_Reported PSC C<...>[74586 rows x 12 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module 'requests' from '<...>\\requests\\__init__.py'>
response            Response     <Response [200]>


## Combine all dataframes into one

In [35]:
# Concatenate the DataFrames
combined_df = pd.concat([df22, df21, df20, df19, df18], ignore_index=True)
combined_df

,Year_Reported,PSC Code,PSC Description,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,2022,D317,IT AND TELECOM- WEB-BASED SUBSCRIPTION,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2022-05-27,COMPETED UNDER SAP,FIRM FIXED PRICE,DATABASE,OTHER FUNCTIONS,RELX INC.,6911482.07
1,2022,B506,SPECIAL STUDIES/ANALYSIS- DATA (OTHER THAN SCI...,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2022-08-04,NOT AVAILABLE FOR COMPETITION,FIXED PRICE LEVEL OF EFFORT,BUSINESS SUPPORT SERVICES,CLOSELY ASSOCIATED,"NORTH WIND INFRASTRUCTURE AND TECHNOLOGY, LLC",10864820.26
2,2022,AN11,HEALTH R&D SERVICES; HEALTH CARE SERVICES; BAS...,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2021-12-20,NOT COMPETED UNDER SAP,COST NO FEE,WORKSHOP ON METHADONE TREATMENT FOR OPIOID USE...,OTHER FUNCTIONS,NATIONAL ACADEMY OF SCIENCES,299163
3,2022,B539,SPECIAL STUDIES/ANALYSIS- AERONAUTICAL/SPACE,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2022-06-15,NOT COMPETED UNDER SAP,LABOR HOURS,TECHNICAL STUDIES AND ANALYTICS SUPPORT SERVICES,OTHER FUNCTIONS,"BRYCE SPACE AND TECHNOLOGY, LLC",726371.02
4,2022,B544,SPECIAL STUDIES/ANALYSIS- TECHNOLOGY,EXECUTIVE OFFICE OF THE PRESIDENT,USA,2022-12-28,NOT COMPETED,COST PLUS FIXED FEE,BUDGET REVIEW,OTHER FUNCTIONS,JOHNS HOPKINS UNIVERSITY APPLIED PHYSICS LABOR...,1200000
...,...,...,...,...,...,...,...,...,...,...,...,...
443763,2018,B510,SPECIAL STUDIES/ANALYSIS- ENVIRONMENTAL ASSESS...,ENVIRONMENTAL PROTECTION AGENCY,USA,2018-05-01,NOT COMPETED,FIRM FIXED PRICE,SITE SURVEY WITH CORE SAMPLING FOR JUAN HORSE ...,NONE,"WESTON SOLUTIONS, INC.",225527.77
443764,2018,F999,OTHER ENVIRONMENTAL SERVICES,ENVIRONMENTAL PROTECTION AGENCY,USA,2018-01-17,NOT AVAILABLE FOR COMPETITION,FIRM FIXED PRICE,"IGF::OT::IGFCALLAHAN MINE O&M CONTRACT, MAINE.",NaN,"CREDERE ASSOCIATES, LLC",183407.06
443765,2018,C219,ARCHITECT AND ENGINEERING- GENERAL: OTHER,ENVIRONMENTAL PROTECTION AGENCY,USA,2018-10-04,FULL AND OPEN COMPETITION,FIRM FIXED PRICE,ARCHITECT ENGINEER PROJECT MANAGEMENT SERVICES...,NaN,"AECOM TECHNICAL SERVICES, INC.",2751982.33
443766,2018,R499,SUPPORT- PROFESSIONAL: OTHER,ENVIRONMENTAL PROTECTION AGENCY,USA,2018-09-14,FULL AND OPEN COMPETITION,FIRM FIXED PRICE,TECHNICAL SUPPORT FOR THE NEW BEDFORD HARBOR S...,NaN,"AECOM TECHNICAL SERVICES, INC.",11183960.43


In [36]:
print(combined_df['Inherently Governmental Functions'].value_counts())


Inherently Governmental Functions
OTHER FUNCTIONS                          289664
CLOSELY ASSOCIATED                        33414
NONE                                      31995
CRITICAL FUNCTIONS                        24808
CLOSELY ASSOCIATED,CRITICAL FUNCTIONS     11113
S                                          3604
E                                           262
E, S                                        105
Name: count, dtype: int64


In [37]:
# Count the number of rows where the 'Inherently Governmental Functions' column value is missing
missing_count = combined_df['Inherently Governmental Functions'].isna().sum()
print("Number of rows with missing Inherently Governmental Functions column value:", missing_count)

Number of rows with missing Inherently Governmental Functions column value: 48803


In [38]:
# Delete all rows where the 'Inherently Governmental Functions' column value is missing
combined_df = combined_df.dropna(subset=['Inherently Governmental Functions'])
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=438
columns_to_drop     list         n=427
columns_to_keep     list         n=11
combined_df         DataFrame    0      Year_Reported PSC <...>394965 rows x 12 columns]
df18                DataFrame    0     Year_Reported PSC C<...>[78116 rows x 12 columns]
df19                DataFrame    0     Year_Reported PSC C<...>[96168 rows x 12 columns]
df20                DataFrame    0      Year_Reported PSC <...>114341 rows x 12 columns]
df21                DataFrame    0     Year_Reported PSC C<...>[74586 rows x 12 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
missing_count       int64        48803
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module 'requests' from '<...>\\requests\\__init

In [39]:
# Count the number of rows where the 'Inherently Governmental Functions' column value is "None"
count_none = (combined_df['Inherently Governmental Functions'] == 'NONE').sum()

print("Number of rows where 'Inherently Governmental Functions' column value is 'NONE':", count_none)

Number of rows where 'Inherently Governmental Functions' column value is 'NONE': 31995


In [40]:
# Delete all rows where the 'Inherently Governmental Functions' column value is 'NONE'
combined_df = combined_df[combined_df['Inherently Governmental Functions'] != 'NONE']
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=438
columns_to_drop     list         n=427
columns_to_keep     list         n=11
combined_df         DataFrame    0      Year_Reported PSC <...>362970 rows x 12 columns]
count_none          int64        31995
df18                DataFrame    0     Year_Reported PSC C<...>[78116 rows x 12 columns]
df19                DataFrame    0     Year_Reported PSC C<...>[96168 rows x 12 columns]
df20                DataFrame    0      Year_Reported PSC <...>114341 rows x 12 columns]
df21                DataFrame    0     Year_Reported PSC C<...>[74586 rows x 12 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
missing_count       int64        48803
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module '

In [41]:
# Save DataFrame to a CSV file
combined_df.to_csv('output.csv', index=False)

In [42]:
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=438
columns_to_drop     list         n=427
columns_to_keep     list         n=11
combined_df         DataFrame    0      Year_Reported PSC <...>362970 rows x 12 columns]
count_none          int64        31995
df18                DataFrame    0     Year_Reported PSC C<...>[78116 rows x 12 columns]
df19                DataFrame    0     Year_Reported PSC C<...>[96168 rows x 12 columns]
df20                DataFrame    0      Year_Reported PSC <...>114341 rows x 12 columns]
df21                DataFrame    0     Year_Reported PSC C<...>[74586 rows x 12 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
missing_count       int64        48803
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests            module       <module '

## Reduce Size of Data File

In [72]:
# Duplicate the data to create indexes
indexed_df = combined_df.copy()

In [73]:
# get info and counts for column data 
#print(indexed_df.info())
print(indexed_df.nunique())

0
Year_Reported                             5
PSC Code                               1417
PSC Description                        1480
Funding Agency Name                     298
Place of Performance Country            186
Date Signed                            3806
Extent Competed                           8
Type of Contract                         13
Description of Requirement           231214
Inherently Governmental Functions         7
Vendor Name                           32904
Total Base and All Options Value     251333
dtype: int64


### Remove decimals from 'Total Base and All Options Value'

In [109]:
# Remove dollar sign, convert to float, truncate decimals, then convert to integer
#indexed_df['Total Base and All Options Value'] = indexed_df['Total Base and All Options Value'].str.split('.').str[0]


In [ ]:
indexed_df.head()

### Index Funding Agency Name

In [74]:
# Not all columns can be indexed
# Columns to index : Funding Agency Name, Vendor Name, Place of Performance Country, Extent Competed, Type of Contract

# Automatically create a mapping dictionary for 'Funding Agency Name'
mapping_dict = {value: str(index + 1) for index, value in enumerate(indexed_df['Funding Agency Name'].dropna().unique())}

# Convert the mapping dictionary into a DataFrame
mapping_df = pd.DataFrame.from_dict(mapping_dict, orient='index', columns=['Name_Index'])

# Save the mapping DataFrame to a CSV file
mapping_df.to_csv('funding_agency_map.csv')

# Display the mapping DataFrame
print(mapping_df)


                                                   Name_Index
EXECUTIVE OFFICE OF THE PRESIDENT                           1
UNITED STATES TRADE AND DEVELOPMENT AGENCY                  2
PEACE CORPS                                                 3
FOREST SERVICE                                              4
US CENSUS BUREAU                                            5
...                                                       ...
CORPORATION FOR NATIONAL AND COMMUNITY SERVICE            294
OFF OF THE DEPUTY UNDER SECRETARY FOR CONGRESSI...        295
OFFICE OF PARTNERSHIPS AND PUBLIC ENGAGEMENT              296
OFFICE FOR CIVIL RIGHTS                                   297
BROADCASTING BOARD OF GOVERNORS                           298

[298 rows x 1 columns]


In [75]:
# Replace values in the 'Funding Agency Name' column with indexed values
indexed_df['Funding Agency Name'] = indexed_df['Funding Agency Name'].map(mapping_dict)
indexed_df.head()

,Year_Reported,PSC Code,PSC Description,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,2022,D317,IT AND TELECOM- WEB-BASED SUBSCRIPTION,1,USA,2022-05-27,COMPETED UNDER SAP,FIRM FIXED PRICE,DATABASE,OTHER FUNCTIONS,RELX INC.,6911482.07
1,2022,B506,SPECIAL STUDIES/ANALYSIS- DATA (OTHER THAN SCI...,1,USA,2022-08-04,NOT AVAILABLE FOR COMPETITION,FIXED PRICE LEVEL OF EFFORT,BUSINESS SUPPORT SERVICES,CLOSELY ASSOCIATED,"NORTH WIND INFRASTRUCTURE AND TECHNOLOGY, LLC",10864820.26
2,2022,AN11,HEALTH R&D SERVICES; HEALTH CARE SERVICES; BAS...,1,USA,2021-12-20,NOT COMPETED UNDER SAP,COST NO FEE,WORKSHOP ON METHADONE TREATMENT FOR OPIOID USE...,OTHER FUNCTIONS,NATIONAL ACADEMY OF SCIENCES,299163
3,2022,B539,SPECIAL STUDIES/ANALYSIS- AERONAUTICAL/SPACE,1,USA,2022-06-15,NOT COMPETED UNDER SAP,LABOR HOURS,TECHNICAL STUDIES AND ANALYTICS SUPPORT SERVICES,OTHER FUNCTIONS,"BRYCE SPACE AND TECHNOLOGY, LLC",726371.02
4,2022,B544,SPECIAL STUDIES/ANALYSIS- TECHNOLOGY,1,USA,2022-12-28,NOT COMPETED,COST PLUS FIXED FEE,BUDGET REVIEW,OTHER FUNCTIONS,JOHNS HOPKINS UNIVERSITY APPLIED PHYSICS LABOR...,1200000


### Index Vendor Name

In [76]:
# Automatically create a mapping dictionary for 'Vendor Name'
mapping_dict = {value: str(index + 1) for index, value in enumerate(indexed_df['Vendor Name'].dropna().unique())}

# Convert the mapping dictionary into a DataFrame
mapping_df = pd.DataFrame.from_dict(mapping_dict, orient='index', columns=['Name_Index'])

# Save the mapping DataFrame to a CSV file
mapping_df.to_csv('vendor_name_map.csv')

# Display the mapping DataFrame
print(mapping_df)

                                                   Name_Index
RELX INC.                                                   1
NORTH WIND INFRASTRUCTURE AND TECHNOLOGY, LLC               2
NATIONAL ACADEMY OF SCIENCES                                3
BRYCE SPACE AND TECHNOLOGY, LLC                             4
JOHNS HOPKINS UNIVERSITY APPLIED PHYSICS LABORA...          5
...                                                       ...
M2C1 INTERNATIONAL, LLC                                 32900
S.T. TURMAN CONTRACTING, L.L.C.                         32901
TURBOMECA USA, INC.                                     32902
OECO, LLC                                               32903
KIRBY MARINA INC                                        32904

[32904 rows x 1 columns]


In [77]:
# Replace values in the 'Vendor Name' column with indexed values
indexed_df['Vendor Name'] = indexed_df['Vendor Name'].map(mapping_dict)
indexed_df.head()

,Year_Reported,PSC Code,PSC Description,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,2022,D317,IT AND TELECOM- WEB-BASED SUBSCRIPTION,1,USA,2022-05-27,COMPETED UNDER SAP,FIRM FIXED PRICE,DATABASE,OTHER FUNCTIONS,1,6911482.07
1,2022,B506,SPECIAL STUDIES/ANALYSIS- DATA (OTHER THAN SCI...,1,USA,2022-08-04,NOT AVAILABLE FOR COMPETITION,FIXED PRICE LEVEL OF EFFORT,BUSINESS SUPPORT SERVICES,CLOSELY ASSOCIATED,2,10864820.26
2,2022,AN11,HEALTH R&D SERVICES; HEALTH CARE SERVICES; BAS...,1,USA,2021-12-20,NOT COMPETED UNDER SAP,COST NO FEE,WORKSHOP ON METHADONE TREATMENT FOR OPIOID USE...,OTHER FUNCTIONS,3,299163
3,2022,B539,SPECIAL STUDIES/ANALYSIS- AERONAUTICAL/SPACE,1,USA,2022-06-15,NOT COMPETED UNDER SAP,LABOR HOURS,TECHNICAL STUDIES AND ANALYTICS SUPPORT SERVICES,OTHER FUNCTIONS,4,726371.02
4,2022,B544,SPECIAL STUDIES/ANALYSIS- TECHNOLOGY,1,USA,2022-12-28,NOT COMPETED,COST PLUS FIXED FEE,BUDGET REVIEW,OTHER FUNCTIONS,5,1200000


### Index Place of Performance Country

In [78]:
# Automatically create a mapping dictionary for 'Vendor Name'
mapping_dict = {value: str(index + 1) for index, value in enumerate(indexed_df['Place of Performance Country'].dropna().unique())}

# Convert the mapping dictionary into a DataFrame
mapping_df = pd.DataFrame.from_dict(mapping_dict, orient='index', columns=['Name_Index'])

# Save the mapping DataFrame to a CSV file
mapping_df.to_csv('country_map.csv')

# Display the mapping DataFrame
print(mapping_df)

    Name_Index
USA          1
PLW          2
BRB          3
NOR          4
LBR          5
..         ...
MUS        182
CUW        183
KNA        184
GRL        185
WSM        186

[186 rows x 1 columns]


In [79]:
# Replace values in the 'Funding Agency Name' column with indexed values
indexed_df['Place of Performance Country'] = indexed_df['Place of Performance Country'].map(mapping_dict)
indexed_df.head()

,Year_Reported,PSC Code,PSC Description,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,2022,D317,IT AND TELECOM- WEB-BASED SUBSCRIPTION,1,1,2022-05-27,COMPETED UNDER SAP,FIRM FIXED PRICE,DATABASE,OTHER FUNCTIONS,1,6911482.07
1,2022,B506,SPECIAL STUDIES/ANALYSIS- DATA (OTHER THAN SCI...,1,1,2022-08-04,NOT AVAILABLE FOR COMPETITION,FIXED PRICE LEVEL OF EFFORT,BUSINESS SUPPORT SERVICES,CLOSELY ASSOCIATED,2,10864820.26
2,2022,AN11,HEALTH R&D SERVICES; HEALTH CARE SERVICES; BAS...,1,1,2021-12-20,NOT COMPETED UNDER SAP,COST NO FEE,WORKSHOP ON METHADONE TREATMENT FOR OPIOID USE...,OTHER FUNCTIONS,3,299163
3,2022,B539,SPECIAL STUDIES/ANALYSIS- AERONAUTICAL/SPACE,1,1,2022-06-15,NOT COMPETED UNDER SAP,LABOR HOURS,TECHNICAL STUDIES AND ANALYTICS SUPPORT SERVICES,OTHER FUNCTIONS,4,726371.02
4,2022,B544,SPECIAL STUDIES/ANALYSIS- TECHNOLOGY,1,1,2022-12-28,NOT COMPETED,COST PLUS FIXED FEE,BUDGET REVIEW,OTHER FUNCTIONS,5,1200000


### Index Extent Competed

In [80]:
# Automatically create a mapping dictionary for 'Extent Competed'
mapping_dict = {value: str(index + 1) for index, value in enumerate(indexed_df['Extent Competed'].dropna().unique())}

# Convert the mapping dictionary into a DataFrame
mapping_df = pd.DataFrame.from_dict(mapping_dict, orient='index', columns=['Name_Index'])

# Save the mapping DataFrame to a CSV file
mapping_df.to_csv('competed_map.csv')

# Display the mapping DataFrame
print(mapping_df)

                                                   Name_Index
COMPETED UNDER SAP                                          1
NOT AVAILABLE FOR COMPETITION                               2
NOT COMPETED UNDER SAP                                      3
NOT COMPETED                                                4
FULL AND OPEN COMPETITION                                   5
FULL AND OPEN COMPETITION AFTER EXCLUSION OF SO...          6
FOLLOW ON TO COMPETED ACTION                                7
COMPETITIVE DELIVERY ORDER                                  8


In [81]:
# Replace values in the 'Extent Competed' column with indexed values
indexed_df['Extent Competed'] = indexed_df['Extent Competed'].map(mapping_dict)
indexed_df.head()

,Year_Reported,PSC Code,PSC Description,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,2022,D317,IT AND TELECOM- WEB-BASED SUBSCRIPTION,1,1,2022-05-27,1,FIRM FIXED PRICE,DATABASE,OTHER FUNCTIONS,1,6911482.07
1,2022,B506,SPECIAL STUDIES/ANALYSIS- DATA (OTHER THAN SCI...,1,1,2022-08-04,2,FIXED PRICE LEVEL OF EFFORT,BUSINESS SUPPORT SERVICES,CLOSELY ASSOCIATED,2,10864820.26
2,2022,AN11,HEALTH R&D SERVICES; HEALTH CARE SERVICES; BAS...,1,1,2021-12-20,3,COST NO FEE,WORKSHOP ON METHADONE TREATMENT FOR OPIOID USE...,OTHER FUNCTIONS,3,299163
3,2022,B539,SPECIAL STUDIES/ANALYSIS- AERONAUTICAL/SPACE,1,1,2022-06-15,3,LABOR HOURS,TECHNICAL STUDIES AND ANALYTICS SUPPORT SERVICES,OTHER FUNCTIONS,4,726371.02
4,2022,B544,SPECIAL STUDIES/ANALYSIS- TECHNOLOGY,1,1,2022-12-28,4,COST PLUS FIXED FEE,BUDGET REVIEW,OTHER FUNCTIONS,5,1200000


### Type of Contract

In [82]:
# Automatically create a mapping dictionary for 'Type of Contract'
mapping_dict = {value: str(index + 1) for index, value in enumerate(indexed_df['Type of Contract'].dropna().unique())}

# Convert the mapping dictionary into a DataFrame
mapping_df = pd.DataFrame.from_dict(mapping_dict, orient='index', columns=['Name_Index'])

# Save the mapping DataFrame to a CSV file
mapping_df.to_csv('contract_type_map.csv')

# Display the mapping DataFrame
print(mapping_df)

                                           Name_Index
FIRM FIXED PRICE                                    1
FIXED PRICE LEVEL OF EFFORT                         2
COST NO FEE                                         3
LABOR HOURS                                         4
COST PLUS FIXED FEE                                 5
FIXED PRICE WITH ECONOMIC PRICE ADJUSTMENT          6
TIME AND MATERIALS                                  7
COST PLUS AWARD FEE                                 8
COST PLUS INCENTIVE FEE                             9
FIXED PRICE INCENTIVE                              10
FIXED PRICE AWARD FEE                              11
COST SHARING                                       12
FIXED PRICE REDETERMINATION                        13


In [83]:
# Replace values in the 'Type of Contract' column with indexed values
indexed_df['Type of Contract'] = indexed_df['Type of Contract'].map(mapping_dict)
indexed_df.head()

,Year_Reported,PSC Code,PSC Description,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,2022,D317,IT AND TELECOM- WEB-BASED SUBSCRIPTION,1,1,2022-05-27,1,1,DATABASE,OTHER FUNCTIONS,1,6911482.07
1,2022,B506,SPECIAL STUDIES/ANALYSIS- DATA (OTHER THAN SCI...,1,1,2022-08-04,2,2,BUSINESS SUPPORT SERVICES,CLOSELY ASSOCIATED,2,10864820.26
2,2022,AN11,HEALTH R&D SERVICES; HEALTH CARE SERVICES; BAS...,1,1,2021-12-20,3,3,WORKSHOP ON METHADONE TREATMENT FOR OPIOID USE...,OTHER FUNCTIONS,3,299163
3,2022,B539,SPECIAL STUDIES/ANALYSIS- AERONAUTICAL/SPACE,1,1,2022-06-15,3,4,TECHNICAL STUDIES AND ANALYTICS SUPPORT SERVICES,OTHER FUNCTIONS,4,726371.02
4,2022,B544,SPECIAL STUDIES/ANALYSIS- TECHNOLOGY,1,1,2022-12-28,4,5,BUDGET REVIEW,OTHER FUNCTIONS,5,1200000


# Create Lookup Table for PSC Codes

In [84]:
# Combine two columns into tuples and convert to dictionary
mapping_dict_psc = dict(zip(indexed_df['PSC Code'], indexed_df['PSC Description']))

In [85]:
# mapping_dict_psc

In [86]:
# Drop the PSC Description column to reduce file size
indexed_df = indexed_df.drop(columns=['PSC Description'])

In [87]:
# Save mapping to .csv file
indexed_df.to_csv('output_indexed.csv', index=False)

### Combine IGF Values 

In [ ]:
# 

## Change column data types

In [ ]:
Vendor Name, 

In [96]:
# Convert 'Date Signed' column to datetime with mixed format
indexed_df['Date Signed'] = pd.to_datetime(indexed_df['Date Signed'], format='mixed', dayfirst=True)

In [101]:
# Convert 'Funding Agency Name' column to integer labels
indexed_df['Funding Agency Name'] = pd.factorize(indexed_df['Funding Agency Name'])[0]

In [103]:
# Convert 'Place of Performance Country' column to integer labels
indexed_df['Place of Performance Country'] = pd.factorize(indexed_df['Place of Performance Country'])[0]

In [105]:
# Convert 'Extent Competed' column to integer labels
indexed_df['Extent Competed'] = pd.factorize(indexed_df['Extent Competed'])[0]

In [107]:
# Convert 'Type of Contract' column to integer labels
indexed_df['Type of Contract'] = pd.factorize(indexed_df['Type of Contract'])[0]

In [110]:
# Convert 'Vendor Name' column to integer labels
indexed_df['Vendor Name'] = pd.factorize(indexed_df['Vendor Name'])[0]

In [113]:
# Convert 'Total Base and All Options Value' column to long integer
indexed_df['Total Base and All Options Value'] = indexed_df['Total Base and All Options Value'].astype(float)

In [118]:
indexed_df.dtypes

0
Year_Reported                                object
PSC Code                                     object
Funding Agency Name                           int64
Place of Performance Country                  int64
Date Signed                          datetime64[ns]
Extent Competed                               int64
Type of Contract                              int64
Description of Requirement                   object
Inherently Governmental Functions            object
Vendor Name                                   int64
Total Base and All Options Value            float64
dtype: object

## Save Indexed df 

In [120]:
# Save Indexed DataFrame to a CSV file
indexed_df.to_csv('output_indexed.csv', index=False)

## Clean up Memory

In [71]:
%whos

Variable            Type         Data/Info
------------------------------------------
BytesIO             type         <class '_io.BytesIO'>
column_names_list   list         n=438
columns_to_drop     list         n=427
columns_to_keep     list         n=11
combined_df         DataFrame    0      Year_Reported PSC <...>362970 rows x 12 columns]
count_none          int64        31995
df18                DataFrame    0     Year_Reported PSC C<...>[78116 rows x 12 columns]
df19                DataFrame    0     Year_Reported PSC C<...>[96168 rows x 12 columns]
df20                DataFrame    0      Year_Reported PSC <...>114341 rows x 12 columns]
df21                DataFrame    0     Year_Reported PSC C<...>[74586 rows x 12 columns]
df22                DataFrame    0     Year_Reported PSC C<...>[80557 rows x 12 columns]
indexed_df          DataFrame    0      Year_Reported PSC <...>362970 rows x 11 columns]
mapping_df          DataFrame                             <...>                  

In [55]:
# clean up data
#del df18
#del df19
#del df20
#del df21
#del df22

## Remove 2018 Data

In [5]:
# Unable to resolve data dictionary discrepancies for 2018 data - decided to remove it from the data set
# Path to  CSV file
file_path = "output_indexed.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df.head()

,Year_Reported,PSC Code,Funding Agency Name,Place of Performance Country,Date Signed,Extent Competed,Type of Contract,Description of Requirement,Inherently Governmental Functions,Vendor Name,Total Base and All Options Value
0,2022,D317,0,0,2022-05-27,0,0,DATABASE,OTHER FUNCTIONS,0,6911482.07
1,2022,B506,0,0,2022-08-04,1,1,BUSINESS SUPPORT SERVICES,CLOSELY ASSOCIATED,1,10864820.26
2,2022,AN11,0,0,2021-12-20,2,2,WORKSHOP ON METHADONE TREATMENT FOR OPIOID USE...,OTHER FUNCTIONS,2,299163.00
3,2022,B539,0,0,2022-06-15,2,3,TECHNICAL STUDIES AND ANALYTICS SUPPORT SERVICES,OTHER FUNCTIONS,3,726371.02
4,2022,B544,0,0,2022-12-28,3,4,BUDGET REVIEW,OTHER FUNCTIONS,4,1200000.00


In [8]:
df.shape

(362970, 11)

In [9]:
# remove 2018 data
# Drop rows where 'Year_Reported' is 2018
df_filtered = df[df['Year_Reported'] != 2018]

In [10]:
df_filtered.shape

(358999, 11)

In [11]:
# Save Indexed DataFrame to a CSV file
df_filtered.to_csv('output_no_2018.csv', index=False)